In [36]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import NoSuchElementException  
from selenium.common.exceptions import NoSuchWindowException 
import time 
from tqdm import tqdm 
from detoxify import Detoxify 
from multiprocessing import Pool

In [19]:
#get url from command line
url = input("Enter the url: ")

#Run driver
driver = webdriver.Chrome('chromedriver')

#Open url
driver.get(url)

Enter the url: https://www.youtube.com/watch?v=mUkTz_1uzSs


In [20]:
while True:
    try:
        comment_section = driver.find_element(By.XPATH, '//*[@id="comments"]')
        driver.execute_script("arguments[0].scrollIntoView();", comment_section)
        scroll = driver.find_element(By.XPATH, '//*[@id="content-text"]')
        driver.execute_script("arguments[0].scrollIntoView();", scroll)
        break
    except NoSuchElementException:
        pass

In [21]:
last_height = driver.execute_script("return document.documentElement.scrollHeight")
ct = 0
while True:
    try:
        driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            ct = ct + 1
            if ct == 5:
                break
        else:
            ct = 0
        last_height = new_height
    except NoSuchWindowException:
        break
try:
    driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
except NoSuchWindowException:
    pass

In [22]:
comments = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
users = driver.find_elements(By.XPATH, '//*[@id="author-text"]/span')

Comments = [comment.text for comment in comments]
Users = [user.text for user in users]

df = pd.DataFrame(list(zip(Users, Comments)), columns=['User', 'Comment'])

In [23]:
driver.close()

In [24]:
df


,User,Comment
0,L D P,the question we should ask ourselfs is : why a...
1,Neil S,This is going to get real weird when an AI pro...
2,Udit Kumar,The fear of the unknown is the greatest fear a...
3,Leo Celista,If they released this kind of stuff openly to ...
4,Krazykov,Ultimately what I would love to see from AI is...
...,...,...
520,Katzenflauscher,I hate the stupid background music. Absolutely...
521,Jim McGowen,Sorry but this is categorically incorrect Jord...
522,DUDE DUDE,"damn, fools in comments think soulless machine..."
523,guess who's back g,HORRIBLE


In [25]:
df.shape

(525, 2)

In [26]:
size = df.shape[0]
size

525

In [40]:
df_negative = pd. DataFrame (columns= ['User','Comment'])
df_positive = pd. DataFrame (columns= ['User','Comment'])

In [ ]:
for i in tqdm(range (size)):
    text = df.at [i, 'Comment']
    
    #detoxify analysis for toxic and abusive comments
    result = Detoxify('original-small').predict(text)
    sentiment_detoxify = max(result.values())
    
    if sentiment_detoxify > 0.60:
        
        df_negative = pd.concat([df_negative, df. loc[i:i]], ignore_index=True)
    else:
        df_positive = pd.concat([df_positive, df.loc[i:i]], ignore_index=True)

In [ ]:
df.to_htm1("Comments.html")
df_positive.to_html("Pos.html")
df_negative.to_html("Neg.html")

In [ ]:
df_negative

In [ ]:
df_positive